# Daily Glucose Overlay

This notebook generates daily glucose overlay charts from blood glucose readings held in the Health Tracker database and retrieved via the Health Tracker Web Service:

- The median line
- A "percentile" chart shading between upper and lower percentiles

This is done for two or more date ranges and the data is plotted for each on the same axes for comparison purposes.

Before attempting to run the notebook:

- Make sure the variables defined in "config.ipynb" are set correctly
- Set the reporting date range and export options in the first code cell

In [ ]:
from datetime import datetime

# Set up the chart definitions for each range consisting of:
#
# Start date
# Number of days
# Chart line colour
# Chart fill colour
# Chart marker
# A list of excluded date ranges
#
# Each excluded date range should be a tuple with a start date and time and end date and time, in that order
chart_definitions = [
    (datetime(2025, 4, 17, 0, 0, 0), 11, "red", "orange", "x", []),
    (datetime(2025, 5, 9, 0, 0, 0), 11, "blue", "blue", "o", []),
    (datetime(2025, 6, 27, 0, 0, 0), 11, "green", "green", "+", []),
    (datetime(2025, 8, 4, 0, 0, 0), 11, "purple", "plum", "*", [(datetime(2025, 8, 6, 6, 52, 0), datetime(2025, 8, 6, 23, 59, 59))])
]

# Export format for the chart:
# PNG     - export as PNG image
# PDF     - export as PDF file
# <blank> - do not export
chart_export_format = "PNG"

In [ ]:
# Define the potential variability in readings
error = 0.082

# Set the Y-axis limits
y_min = 3
y_max = 15

In [ ]:
%run ../api.ipynb
%run ../config.ipynb
%run ../export.ipynb

In [ ]:
from datetime import timedelta
import numpy as np

# Log in to the service and get the person ID
token = authenticate(url, username, password)
person_id = get_person_id(url, token, firstnames, surname)

# Iterate over each definition
xticks_to_show = None
agp = []
for definition in chart_definitions:
    # Calculate the end date
    start_date, days, _, _, _, exclusions = definition
    end_date = start_date + timedelta(days=days)

    # Load the data
    df = get_blood_glucose_measurements(url, token, person_id, start_date, end_date)
    df = remove_records_for_date_ranges(df, exclusions)
    if not df.empty:
        # Rename the existing date column, that contains a date and time and round the timestamp to the
        # nearest 5 minutes to align the data (failing to do this means the pivot won"t work well when
        # plotted)
        df.rename(columns={ "date": "timestamp" }, inplace=True)
        df["timestamp"] = df["timestamp"].dt.round("5min")

        # Create columns for date and time-of-day
        df["date"] = df["timestamp"].dt.date
        df["time"] = df["timestamp"].dt.strftime("%H:%M")

        # Pivot the data where rows are the time of day and columns are the date
        pivot_df = df.pivot(index="time", columns="date", values="level")

        # Set x-ticks to every hour (12 x 5 minutes = 60 minutes = 1 hour). This is done once, here, ahead
        # of plotting, below
        if xticks_to_show is None:
            xticks_to_show = pivot_df.index[::12]

        # Calculate the median line
        median = pivot_df.median(axis=1)

        # # Calculate percentile bands for the shaded plot
        upper = median * (1.0 + error)
        lower = median * (1.0 - error)

        # Add the data to the collection for plotting
        agp.append((median, lower, upper))
    else:
        agp.append((None, None, None))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# Create the chart
plt.figure(figsize=(12, 8))

# Iterate over each definition
for i, definition in enumerate(chart_definitions):
    # Extract the chart properties
    start_date, days, line_colour, fill_colour, marker, _ = definition
    end_date = start_date + timedelta(days=days)
    label = f"{start_date.strftime('%d-%b-%Y')} to {end_date.strftime('%d-%b-%Y')}"

    # Get the data for median line and lower and upper bands
    median, lower, upper = agp[i]
    if median is not None:
        # Shade the error range
        plt.fill_between(median.index, lower, upper, color=fill_colour, alpha=0.1)

        # Draw the median line
        plt.plot(median.index, median.values, color=line_colour, label=label, linewidth=1)

# Add title and labels
plt.title("Daily Median Glucose Overlay")
plt.xlabel("Time of Day")
plt.ylabel("Glucose (mmol/L)")
plt.ylim(y_min, y_max)
plt.xticks(ticks=xticks_to_show, rotation=90)
plt.grid(True)

# Create a legend_range_one below the plot
plt.legend(
    title="Date",
    bbox_to_anchor=(0.5, -0.2),
    loc="upper center",
    ncol=6,
    fontsize="small",
    title_fontsize="small"
)

plt.tight_layout()

# Export to PNG or PDF, if required
export_chart("glucose_overlay_comparison", "Percentile", chart_export_format)

plt.show()
